In [7]:
import pandas as pd

# Carregar dados da camada silver
df_clean = pd.read_parquet('data/silver/dados_limpos.parquet')

# Linhas stockcode de tarifas
stockcode_fees = ['C2', 'DOT', 'POST','AMAZONFEE']
df_clean['total_value'] = df_clean['Quantity'] * df_clean['UnitPrice']

# =========================================
#               CRIAÇÃO DE TABELAS
# =========================================

# --------- Dimensao Country -----------
dim_country = (
    df_clean[['Country']]
    .drop_duplicates()
    .copy()
)
dim_country = dim_country.reset_index(drop=True)
dim_country['CountryID'] = dim_country.index + 1

# --------- Dimensao Date -----------
dim_date = (
    df_clean[['InvoiceDate']]
    .drop_duplicates(subset=['InvoiceDate'])
    .copy()
)
dim_date = dim_date.reset_index(drop=True)
dim_date['InvoiceDate'] = pd.to_datetime(dim_date['InvoiceDate'])
dim_date['DateID'] = dim_date.index + 1
dim_date['Year'] = dim_date['InvoiceDate'].dt.year
dim_date['Month'] = dim_date['InvoiceDate'].dt.month
dim_date['Day'] = dim_date['InvoiceDate'].dt.day
dim_date['Weekday'] = dim_date['InvoiceDate'].dt.day_name()
dim_date['Hour'] = dim_date['InvoiceDate'].dt.hour

# Mapeamentos para substituição
country_map = dict(zip(dim_country['Country'], dim_country['CountryID']))
date_map = dict(zip(dim_date['InvoiceDate'], dim_date['DateID']))

# --------- Fato Vendas -----------
fact_sales = df_clean[
    (~df_clean['InvoiceNo'].astype(str).str.startswith(('C', 'A'))) &
    (df_clean['Quantity'] > 0) &
    (df_clean['UnitPrice'] > 0) &
    (~df_clean['StockCode'].isin(stockcode_fees))
].copy()

# Converter CustomerID para string
fact_sales['CustomerID'] = fact_sales['CustomerID'].fillna('Unknown').astype(str)
fact_sales['CountryID'] = fact_sales['Country'].map(country_map)
fact_sales['DateID'] = pd.to_datetime(fact_sales['InvoiceDate']).map(date_map)

fact_sales = fact_sales[[
    'InvoiceNo', 'StockCode', 'CustomerID', 'DateID',
    'Quantity', 'UnitPrice', 'total_value', 'CountryID'
]]

# --------- Fato Tarifas -----------
fact_fees = df_clean[
    (df_clean['StockCode'].isin(stockcode_fees)) &
    (~df_clean['InvoiceNo'].astype(str).str.startswith(('C', 'A')))].copy()
fact_fees['CustomerID'] = fact_fees['CustomerID'].astype(str)
fact_fees['CountryID'] = fact_fees['Country'].map(country_map)
fact_fees['DateID'] = pd.to_datetime(fact_fees['InvoiceDate']).map(date_map)
fact_fees = fact_fees[[
    'InvoiceNo', 'StockCode', 'CustomerID', 'DateID',
    'Quantity', 'UnitPrice', 'total_value', 'CountryID'
]]

# --------- Fato Cancelamentos -----------
fact_cancellations = df_clean[
    (df_clean['InvoiceNo'].astype(str).str.startswith(('C', 'A'))) &
    (~df_clean['StockCode'].isin(['C2', 'DOT', 'POST']))
].copy()
fact_cancellations['CustomerID'] = fact_cancellations['CustomerID'].astype(str)
fact_cancellations['CountryID'] = fact_cancellations['Country'].map(country_map)
fact_cancellations['DateID'] = pd.to_datetime(fact_cancellations['InvoiceDate']).map(date_map)
fact_cancellations = fact_cancellations[[
    'InvoiceNo', 'StockCode', 'CustomerID', 'DateID',
    'Quantity', 'UnitPrice', 'total_value', 'CountryID'
]]

# =========================================
#               TABELAS DIMENSAO
# =========================================

# --------- Dim Clientes -----------
all_customers = pd.concat([
    fact_sales[['CustomerID', 'CountryID']],
    fact_fees[['CustomerID', 'CountryID']],
    fact_cancellations[['CustomerID', 'CountryID']]
], ignore_index=True)

dim_customer = (
    all_customers
    .drop_duplicates(subset=['CustomerID'])
    .copy()
)
dim_customer = dim_customer.reset_index(drop=True)

# Garante existencia do cliente 'Unknown'
if 'Unknown' not in dim_customer['CustomerID'].values:
    unknown_country_id = dim_country[dim_country['Country'] == 'Desconhecido']['CountryID'].values[0] if 'Desconhecido' in dim_country['Country'].values else None
    dim_customer.loc[len(dim_customer)] = ['Unknown', unknown_country_id]

# --------- Dim Produtos -----------
dim_product = (
    df_clean[['StockCode', 'Description']]
    .drop_duplicates(subset=['StockCode'])
    .rename(columns={'Description': 'ProductDescription'})
    .copy()
)

# --------- Dim Date -----------
# Já criada acima como dim_date, apenas reordenar colunas
dim_date = dim_date[[
    'DateID', 'InvoiceDate', 'Year', 'Month', 'Day', 'Weekday', 'Hour'
]]

# --------- Dim Country -----------
dim_country = dim_country[['CountryID', 'Country']]



In [8]:
# Classificando vendas/tarifas/cancelamentos
fact_sales['TransactionType'] = 'Sale'
fact_fees['TransactionType'] = 'Fee'
fact_cancellations['TransactionType'] = 'Cancellation'

# Unindo tabelas fatos em uma só
fact_all = pd.concat(
    [fact_sales, fact_fees, fact_cancellations],
    ignore_index=True
)

In [9]:
# =========================================
#            MÉTRICAS GOLD
# =========================================
# ----------- Análise RFM -----------

# Junte fact_sales com dim_date para obter a data real
fact_sales = fact_sales.merge(
    dim_date[['DateID', 'InvoiceDate']],
    on='DateID',
    how='left'
)

# Garanta que InvoiceDate está em datetime
fact_sales['InvoiceDate'] = pd.to_datetime(fact_sales['InvoiceDate'])

# Última data de compra
latest_date = fact_sales['InvoiceDate'].max()

# RFM com InvoiceDate
rfm = (
    fact_sales.groupby('CustomerID')
    .agg(
        Recency=('InvoiceDate', lambda x: (latest_date - x.max()).days),
        Frequency=('InvoiceNo', 'nunique'),
        Monetary=('total_value', 'sum')
    )
    .reset_index()
)

# ----------- Análise de produtos ----------- 

# Mais vendido por quantidade
most_quantity_product = (
    fact_sales.groupby(['CustomerID', 'StockCode'])
    .agg(UnitsSold=('Quantity', 'sum'))
    .reset_index()
    .sort_values(['CustomerID', 'UnitsSold'], ascending=[True, False])
    .drop_duplicates('CustomerID')
)

# Mais vendido por transação
most_transaction_product = (
    fact_sales.groupby(['CustomerID', 'StockCode'])
    .agg(Transactions=('InvoiceNo', 'nunique'))
    .reset_index()
    .sort_values(['CustomerID', 'Transactions'], ascending=[True, False])
    .drop_duplicates('CustomerID')
)

# Combinando resultados
most_purchased_products = most_quantity_product.merge(
    most_transaction_product,
    on=['CustomerID', 'StockCode']
)

# ----------- Metricas gerais -----------

# Vendas bruta
gross_sales = fact_sales['total_value'].sum()

# Vendas liquidas
net_sales = gross_sales + fact_cancellations['total_value'].sum() - fact_fees['total_value'].sum()

# Total de pedidos
metric_total_orders = fact_sales['InvoiceNo'].nunique()

# Clientes distintos
distinct_customers = dim_customer['CustomerID'].nunique()

# Produtos distintos
distinct_products = dim_product['StockCode'].nunique()

# Unidades vendidas
product_units_sold = fact_sales['Quantity'].sum()

In [10]:
# ------- visualizar metricas ------- 
import pandas as pd

# Exibir RFM e produtos mais comprados
display(rfm)
display(most_purchased_products)

# Exibir métricas gerais em tabela
metrics = pd.DataFrame({
    'Metrics': [
        'Gross Sales',
        'Net Sales',
        'Total Orders',
        'Distinct Customers',
        'Distinct Products',
        'Product Units Sold'
    ],
    'Value': [
        gross_sales,
        net_sales,
        metric_total_orders,
        distinct_customers,
        distinct_products,
        product_units_sold
    ]
})

display(metrics)

,CustomerID,Recency,Frequency,Monetary
0,12346.0,325,1,77183.60
1,12347.0,1,7,4310.00
2,12348.0,74,4,1437.24
3,12349.0,18,1,1457.55
4,12350.0,309,1,294.40
...,...,...,...,...
4334,18281.0,180,1,80.82
4335,18282.0,7,2,178.05
4336,18283.0,3,16,2094.88
4337,18287.0,42,3,1837.28


,CustomerID,StockCode,UnitsSold,Transactions
0,12346.0,23166,74215,1
1,12348.0,23077,200,2
2,12358.0,15056BL,36,2
3,12359.0,22423,27,3
4,12362.0,22629,60,5
...,...,...,...,...
948,18269.0,21731,24,1
949,18273.0,79302M,80,3
950,18274.0,21108,18,1
951,18280.0,22084,8,1


,Metrics,Value
0,Gross Sales,1.035046e+07
1,Net Sales,9.149362e+06
2,Total Orders,1.987800e+04
3,Distinct Customers,4.373000e+03
4,Distinct Products,4.070000e+03
5,Product Units Sold,5.584375e+06


In [11]:
# Salvar tabelas finais (camada Gold)
gold_path = 'data/gold/'

dim_country.to_parquet(f'{gold_path}dim_country.parquet', index=False)
dim_date.to_parquet(f'{gold_path}dim_date.parquet', index=False)
dim_customer.to_parquet(f'{gold_path}dim_customer.parquet', index=False)
dim_product.to_parquet(f'{gold_path}dim_product.parquet', index=False)
fact_all.to_parquet(f'{gold_path}fact_all.parquet', index=False)
rfm.to_parquet(f'{gold_path}rfm.parquet', index=False)
most_purchased_products.to_parquet(f'{gold_path}most_purchased_products.parquet', index=False)
metrics.to_parquet(f'{gold_path}metrics.parquet', index=False)

print("Tabelas salvas na camada Gold.")

Tabelas salvas na camada Gold.


In [12]:
display(fact_sales['total_value'].sum())

np.float64(10350459.744)